In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
import librosa
import numpy as np    
import os
import signal
import subprocess
import json
import uuid
from IPython.display import HTML

Extractor (Input Validation, Chunking, Mapping From file_csv and annotations_csv to chunks)

In [3]:
from acoustic_knowledge_discovery import extractors

In [4]:
extractor = extractors.Extractor(
    "sample_inputs/file.csv", #file metadata csv
    #"/home/s.kamboj.400/mount/gorongosa_recordings_small_test.csv",
    #"/home/s.kamboj.400/mount/mexico_recordings.csv",
    "sample_inputs/anno.csv", #annoations csv
    #"/home/s.kamboj.400/mount/gorongosa_recordings_anno.csv",
    #"/home/s.kamboj.400/mount/mexico_recordings_anno.csv",
    "sample_inputs/inputDtory",#input directory with a files subdirectory for audio files & a XC-templates subdirectory for XC templates
    #"/home/s.kamboj.400/mount/",
    #"/home/s.kamboj.400/unzipped-coral/PaolaMexico/",
    chunk_size = 5) #chunk size in seconds
chunkDS = extractor.forward()


Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

In [8]:
for row in chunkDS.chunk_ds["train"]:
    print(row)
chunkDS.chunk_ds

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'file_path': '/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory/files/XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', 'chunk_start': 0, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'file_path': '/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory/files/XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', 'chunk_start': 5, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - 

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence'],
        num_rows: 4
    })
})

Chunk DS Augmentations
- NOTE: Old because we are now computing on the fly not storing in memory

In [ ]:
# def add_raw_audio_batched(batch):
#     raws = []
#     for path, start in zip(batch["file_path"], batch["chunk_start"]):
#         # os.fspath handles both str and Path
#         #print("os.fspath(path) is ", os.fspath(path))
#         y, _ = librosa.load(
#             os.fspath(path),
#             offset=float(start),
#             duration=float(extractor.chunk_size),
#             sr=None         # keep original sampling rate     
#         )
#         raws.append(y.astype(np.float32))

#     return {"raw_audio": raws}


In [ ]:
# # Add raw audio column in dataset
# chunkDS.chunk_ds = chunkDS.chunk_ds.map(add_raw_audio_batched, 
#                                         batched=True, 
#                                         num_proc=1) #increasing this number a small amount can make it faster
#                                         # ONLY INCREASE ON MAC OR LINUX

In [ ]:
# chunkDS.chunk_ds #make sure that new column raw_audio is added

File-level Operations

In [ ]:
from acoustic_knowledge_discovery.features.feature_preprocessing_pipeline import FeatureSequential
from acoustic_knowledge_discovery import features

THRESHOLD = 0.6

pipeline = FeatureSequential(
    features.TemplateMatching(
        extractor.base_dir / "XC-templates", # link to directory with XC templates
        THRESHOLD,
        chunk_size=extractor.chunk_size
    ),
    # features.anotherFunctionThatInheretsFromFeaturePreprocessor
)

chunkDS = pipeline.forward(chunkDS)  

Number of files to process:  4
Number of file_names 4
1 total templates
Filtered down to 1 unique species templates
  XC758442 - Common Blackbird - Turdus merula: XC758442 - Common Blackbird - Turdus merula.wav


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Progress: 1/1 templates completed (100.0%)
Total matches found above the threshold of 0.6: 1


In [14]:
#verify properly loaded after feature extraction
for row in chunkDS.chunk_ds["train"]:
    #print(row["Annotation"], row["Confidence"])
    print(row)

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'file_path': '/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory/files/XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', 'chunk_start': 0, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', ' time_of_day': 'Morning', 'season': 'Rainy', 'species': ['Bird', 'Bat'], 'SR': 48000.0, 'file_path': '/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory/files/XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', 'chunk_start': 5, 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5', 'Annotation': ['owl'], 'Confidence': [0.8999999761581421]}
{'file_name': 'XC233970 - Black-chinned Sparrow - 

Chunk-level Operations

In [15]:
#Insert EGCI
from acoustic_knowledge_discovery import features

egci = features.EGCI()
chunkDS= egci(chunkDS, extractor.chunk_size, num_proc=1)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [16]:
#verify EGCI column properly added
# for row in chunkDS.chunk_ds["train"]:
#     print(row["chunk_id"], row["EGCI"])
chunkDS.chunk_ds

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI'],
        num_rows: 4
    })
})

Binning!

In [17]:
from acoustic_knowledge_discovery.postprocessing import MakeBins2dFloat
makebins = MakeBins2dFloat(chunkDS)
egci_bin={
    "low": ( (0.0, 34), (0.0,0.34) ), 
    "medium": ((0.34,0.67),(0.34,0.67)), 
    "high": ((0.67,1),(0.67,1))
}

chunkDS = makebins("EGCI", egci_bin)

Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [18]:
for row in chunkDS.chunk_ds["train"]:
    #label_str = chunkDS.chunk_ds["train"].features["EGCI"].int2str(row["EGCI"])
    print(row["chunk_id"], row["EGCI"])
chunkDS.chunk_ds

XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0 Other
XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_5 low
XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_10 medium
XC758442 - Common Blackbird - Turdus merula.wav_0 Other


DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI'],
        num_rows: 4
    })
})

In [19]:
chunkDS.chunk_ds["train"][0]

{'file_name': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3',
 ' time_of_day': 'Morning',
 'season': 'Rainy',
 'species': ['Bird', 'Bat'],
 'SR': 48000.0,
 'file_path': '/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory/files/XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3',
 'chunk_start': 0,
 'chunk_id': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0',
 'Annotation': ['owl'],
 'Confidence': [0.8999999761581421],
 'EGCI': 'Other'}

In [ ]:
# # Tests that MakeBins1dFloat does work, but it does not make sense to use it on this dataset
# from acoustic_knowledge_discovery.postprocessing import MakeBins1dFloat
# makebins1d = MakeBins1dFloat(chunkDS)
# OneDBin={
#     "low": (0, 5) , 
#     "medium": (5, 10), 
#     "high": (10, 15)
# }

# chunkDS = makebins1d("chunk_start", OneDBin)

# for row in chunkDS.chunk_ds["train"]:
#     print(row["chunk_id"], row["chunk_start"])
# chunkDS.chunk_ds

# AutoEncoder and Clustering

In [20]:
from acoustic_knowledge_discovery.features import AutoEncoderProcessor

autoEncoder = AutoEncoderProcessor(model_path = "sample_models/dim_10_muha.pt", num_dims=10)
chunkDS = autoEncoder(chunkDS, extractor.chunk_size)
chunkDS.chunk_ds

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings'],
        num_rows: 4
    })
})

In [21]:
from acoustic_knowledge_discovery.postprocessing import Cluster_API

#HEY HEY HEY: You should note that more num_clusters means more possible nodes
# num_clusters should be some number that is less than your dataset
clustering_pipe = Cluster_API(chunkDS, num_clusters=2)
chunkDS = clustering_pipe("sample_models/dim_10_muha.pt_embeddings")
chunkDS.chunk_ds

DatasetDict({
    train: Dataset({
        features: ['file_name', ' time_of_day', 'season', 'species', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings', 'sample_models/dim_10_muha.pt_embeddings_clusters'],
        num_rows: 4
    })
})

# Convert System to a graph representation

In [22]:
def normalize(v):
    if isinstance(v, list):
        return v
    if isinstance(v, np.ndarray):   # convert numpy array to list
        return v.tolist()
    return [v]  # if it is scalar, then make it list

In [25]:
#test = chunkDS.chunk_ds.select_columns(["file_name", " time_of_day", "season", "species", "chunk_id", "Annotation", "EGCI", "sample_models/dim_10_muha.pt_embeddings_clusters"])
#TODO : change the column names to match dataset
test = chunkDS.chunk_ds.select_columns(["file_name", "season"," time_of_day", "chunk_id", "EGCI", "sample_models/dim_10_muha.pt_embeddings_clusters"])
nodes = test["train"].to_pandas().melt()
nodes = nodes.assign(value=nodes["value"].apply(normalize)).explode("value", ignore_index=True) #necessary so list values are separated
nodes["id"] = nodes["variable"] + "_" + nodes["value"].apply(str)
nodes["group"] = nodes["variable"].apply(lambda x: list(nodes["variable"].unique()).index(x))
nodes[["id", "group"]].head()

,id,group
0,file_name_XC233970 - Black-chinned Sparrow - S...,0
1,file_name_XC233970 - Black-chinned Sparrow - S...,0
2,file_name_XC233970 - Black-chinned Sparrow - S...,0
3,file_name_XC758442 - Common Blackbird - Turdus...,0
4,season_Rainy,1


In [26]:
edge_list = test["train"].to_pandas().melt(id_vars="file_name")
edge_list = edge_list.assign(value=edge_list["value"].apply(normalize)).explode("value", ignore_index=True)
edge_list = edge_list.dropna(subset=["value"])# drop Nan
edge_list["target"] = edge_list["variable"] + "_" + edge_list["value"].apply(str)
edge_list["source"] = "file_name" + "_" + edge_list["file_name"]
edge_list["value"] = 1
edge_list[["source", "target", "value"]].head()

,source,target,value
0,file_name_XC233970 - Black-chinned Sparrow - S...,season_Rainy,1
1,file_name_XC233970 - Black-chinned Sparrow - S...,season_Rainy,1
2,file_name_XC233970 - Black-chinned Sparrow - S...,season_Rainy,1
3,file_name_XC758442 - Common Blackbird - Turdus...,season_Dry,1
4,file_name_XC233970 - Black-chinned Sparrow - S...,time_of_day_Morning,1


In [27]:
import json

with open("acoustic_knowledge_discovery/d3-visualization/graph_representation.json", mode="w") as f:
    json.dump({
        "nodes": list(nodes[["id", "group"]].drop_duplicates().T.to_dict().values()),  
        "links":  list(edge_list[["source", "target", "value"]].T.to_dict().values()),
    }, f
)

# Visualization

In [29]:
!npm install

m#########⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠼ idealTree: timing idealTree Completed in 36ms
up to date, audited 13 packages in 344ms

found 0 vulnerabilities


1. In terminal, type ` cd acoustic_knowledge_discovery/d3-visualization/ `, assuming that you are in the acoustic_knowledge_discovery parent directory
2. Run ` npx http-server -p 8080 ` in your terminal*
3. Open ` http://127.0.0.1:8080 ` in Safari/Chrome/your choice of browser to interact with the visualization.


*NOTE: If you are getting the error that npx does not exist, then download it here: https://nodejs.org/en/download

The following 3 lines of code open up the visualization in the jupyter notebook, but it currently only works for MacOS users. 

In [ ]:
# command = ["npx", "http-server", "-p", "8080"] 
# target_directory = "acoustic_knowledge_discovery/d3-visualization/"

# process = subprocess.Popen(
#     command,
#     cwd=target_directory,
#     #shell=True,
#     preexec_fn=os.setsid  
# )

# # Persist the group leader PID so you can kill it later even after kernel restarts & kill the entire tree
# with open("/tmp/http_server_pid.json", "w") as f:
#     json.dump({"pgid": process.pid}, f)

# print(f"Server started. PGID={process.pid}")

: 

In [ ]:
# #NOTE : You can also open up http://127.0.0.1:8080 in your browser to see it displayed on a larger screen
# HTML(f"""
# <iframe style="background-color: white;" 
#         src="http://127.0.0.1:8080/?nocache={uuid.uuid4()}" 
#         width="1200" height="1000"></iframe>
# """)

: 

In [ ]:
# # Kill the entire process group (gracefully) to stop d3vis port
# with open("/tmp/http_server_pid.json") as f:
#     pgid = json.load(f)["pgid"]


# os.killpg(pgid, signal.SIGTERM)

: 

In [ ]:
#process.kill()

: 

: 